In [10]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('trojanwar.pdf')
docs=loader.load()
docs

[Document(page_content='The \nStory of  \nthe Trojan  \nwar \nLong ago, when the gods still walked the earth, a great feast was held to celebrate \nthe wedding of Thetis and Peleus. All the gods were invited except Eris, the \ngoddess of quarrels. The wedding feast  was almost over when Eris stormed in. \nShe threw a golden apple on the table. On it were written the words: “For the \nfairest.” Each of the goddesses thought she was the fairest and should have the \napple. All of them were very beautiful. Nobody could decide  between them. They \nargued about it for years. At last the gods chose the three loveliest, and they \ndecided to ask a human to judge between them. They chose Paris, the son of the \nking of Troy. He was very surprised one evening to see Hermes appear with thr ee \ngoddesses. They were: Hera, the queen of the gods; Athena, the goddess of \nwisdom; and Aphrodite, the goddess of love and beauty. “Congratulations, Paris!” \nsaid Hermes. “You have been chosen to judge 

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunk_documents=text_splitter.split_documents(docs)
chunk_documents

[Document(page_content='The \nStory of  \nthe Trojan  \nwar \nLong ago, when the gods still walked the earth, a great feast was held to celebrate \nthe wedding of Thetis and Peleus. All the gods were invited except Eris, the \ngoddess of quarrels. The wedding feast  was almost over when Eris stormed in. \nShe threw a golden apple on the table. On it were written the words: “For the \nfairest.” Each of the goddesses thought she was the fairest and should have the \napple. All of them were very beautiful. Nobody could decide  between them. They \nargued about it for years. At last the gods chose the three loveliest, and they \ndecided to ask a human to judge between them. They chose Paris, the son of the \nking of Troy. He was very surprised one evening to see Hermes appear with thr ee \ngoddesses. They were: Hera, the queen of the gods; Athena, the goddess of \nwisdom; and Aphrodite, the goddess of love and beauty. “Congratulations, Paris!” \nsaid Hermes. “You have been chosen to judge 

In [12]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS


db = FAISS.from_documents(
    chunk_documents,
    OllamaEmbeddings(model='nomic-embed-text')
)
db

In [15]:
query = """At last Achilles slashed Hector across the
throat. Hector fell to the ground dying"""
retrieved_result= db.similarity_search(query)
print(retrieved_result[0].page_content)

sword and rushed out of the tent. Outside the gates of  Troy stood a figure in 
golden armour . Achilles hacked his way  through the ranks of Trojans until he 
reached Hector. They  fought all day. At last Achilles slashed Hector across the  
throat. Hector fell to the ground dying. With his last breath he  begged Achilles to 
return his body to his famil y so that they  could have a proper funeral and release 
his soul. But Achilles  tied Hector’s body to the back of his chariot and drove round  
the walls of Troy three times a day. Not only was Hector dead,  but his family 
couldn’t lay his soul to rest. At last  the king of  Troy, Hector’s father, could bear it 
no longer. He slipped out of  Troy ar night and crept up the hill to Achilles’s tent. 
He talked  about Achilles’s own father and how he would feel if Achilles  had been 
killed.  “I will ransom Hector’s body to you for its weight in gold,” said  Achilles.


In [16]:
#Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(""" 
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>
Question:{input}

""")

In [17]:
from langchain_community.llms import Ollama
##Ollama llama2
llm=Ollama(model="llama2")
llm

Ollama()

In [18]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template=' \nAnswer the following question based only on the provided context.\nThink step by step before providing a detailed answer.\n<context>\n{context}\n</context>\nQuestion:{input}\n\n'))])
| Ollama()
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [19]:
retriever= db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002046D0FC070>)

In [20]:
from langchain.chains import create_retrieval_chain
retrieval_chain= create_retrieval_chain(retriever,document_chain)

In [21]:
response= retrieval_chain.invoke({"input":"What did Achilles ask to give Hector's body?"})
response['answer']

"Based on the provided context, Achilles asked for Hector's body to be ransomed for its weight in gold."

In [23]:
response= retrieval_chain.invoke({"input":"Who is the son of thetis"})
response['answer']

'The answer to your question can be found in the context provided:\n\n"The best fighter, Achilles, was the son of Thetis and Peleus."\n\nTherefore, the son of Thetis is Achilles.'